In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from nltk.stem import WordNetLemmatizer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
trainData = pd.read_json("../input/train.json")
ingredientNames = set([names.lower() for lists in trainData['ingredients'] for names in lists])
ingredientToId = {}
for name in ingredientNames:
    if not name in ingredientToId:
        ingredientToId[name] = len(ingredientToId)
xTrain = []

for entry in trainData['ingredients']:
    oneHot = [0] * len(ingredientToId)
    for name in entry:
        if name.lower() in ingredientToId:
            ingredientId = ingredientToId[name.lower()]
            oneHot[ingredientId] += 1
    xTrain.append(oneHot)

yTrain = []
cuisines = set(trainData['cuisine'])
cuisineToId = {}
for cuisine in cuisines:
    if not cuisine in cuisineToId:
        cuisineToId[cuisine] = len(cuisineToId)
for cuisine in trainData['cuisine']:
    oneHot = [0] * len(cuisineToId)
    if cuisine in cuisineToId:
        cuisineId = cuisineToId[cuisine]
        oneHot[cuisineId] += 1
    yTrain.append(oneHot)
    
        
    


# Any results you write to the current directory are saved as output.

In [ ]:
from keras.models import  Sequential
from keras.layers.core import  Lambda , Dense, Activation, Flatten, Dropout, Reshape
from keras import regularizers

print("Starting training")
model = Sequential()
model.add(Dense(40, input_dim = len(ingredientNames), kernel_regularizer=regularizers.l2(0.000005)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(20))

model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

model.fit(xTrain, yTrain, epochs=10, batch_size=16, validation_split=0.2)